# 科学技術計算 4 レポート


In [4]:
import math
import cmath
import numpy as np
import scipy
from typing import Tuple
from tqdm.auto import tqdm

rng = np.random.default_rng()

## 課題04-1

In [40]:
def lu_decomposition( A_org: np.ndarray ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A_org.ndim == 2
    assert A_org.shape[0] == A_org.shape[1]

    #前進消去を行うコード
    n = A_org.shape[0]
    A = A_org.copy()
    print("A\n", A, sep="")
    L = np.eye(n)   #単位行列生成
    P = np.eye(n)   #置換行列

    for j in range(n - 1):
        #ここから資料にあった部分ピポッド選択月も前進消去の実装
          
        print(f"{j}-th row", A[j])
        
        # 部分ピボット選択
        j_max = np.abs(A[j:, j]).argmax() + j   #行野中の一番大きい値を抽出
        if j != j_max:
            # A の行交換
            A[[j, j_max]] = A[[j_max, j]]

            # L の行交換（前の列のみ）
            if j > 0:
                L[[j, j_max], :j] = L[[j_max, j], :j]

            # P の行交換 ← これが scipy と同じ P を作るカギ
            P[[j, j_max]] = P[[j_max, j]]
        
        print("A\n", A, sep="")
        print("P\n", P)
    
        # 前進消去
        remaining_rows = n - (j + 1)
        r_ij = A[j + 1:, j] / A[j, j]
        A[j + 1:, j + 1:] -= np.tile(A[j, j + 1:], (remaining_rows, 1)) * r_ij.reshape(remaining_rows, -1)
        A[j + 1:, j] = 0
        L[j + 1:, j] = r_ij
        print("A\n", A, sep="")
        print("L\n", L, sep="")
        
    U = A
    
    print("U\n", U, sep="")


    #確認１
    print("P =",P)
    print("L =", L)
    print("U =", U)

    print("LU\n", L @ U, sep="")
    print("A\n", A_org, sep="")
    #print("LU == A", np.allclose(L @ U, A_org))
    
    return P, L, U

A = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])



for i in tqdm(range(50)):
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)

    print("Psp = ", Psp)
    print("Lsp = ", Lsp)
    print("Usp = ", Usp)

    assert np.allclose(P @ A, L @ U)
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"


  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.25404612 0.10992196]
 [0.14017944 0.39644687]]
0-th row [0.25404612 0.10992196]
A
[[0.25404612 0.10992196]
 [0.14017944 0.39644687]]
P
 [[1. 0.]
 [0. 1.]]
A
[[0.25404612 0.10992196]
 [0.         0.33579332]]
L
[[1.         0.        ]
 [0.55178737 1.        ]]
U
[[0.25404612 0.10992196]
 [0.         0.33579332]]
P = [[1. 0.]
 [0. 1.]]
L = [[1.         0.        ]
 [0.55178737 1.        ]]
U = [[0.25404612 0.10992196]
 [0.         0.33579332]]
LU
[[0.25404612 0.10992196]
 [0.14017944 0.39644687]]
A
[[0.25404612 0.10992196]
 [0.14017944 0.39644687]]
Psp =  [[1. 0.]
 [0. 1.]]
Lsp =  [[1.         0.        ]
 [0.55178737 1.        ]]
Usp =  [[0.25404612 0.10992196]
 [0.         0.33579332]]
A
[[0.53730428 0.77147232 0.1885836  0.60607743 0.14596521 0.15592645
  0.81329713]
 [0.74570637 0.20294285 0.7638467  0.99455543 0.6489372  0.77653392
  0.39335665]
 [0.70888549 0.68499788 0.06737669 0.56059102 0.75002801 0.08671539
  0.7946971 ]
 [0.53533152 0.68279621 0.07619263 0.39238679 0.91

## 課題04-2


## 課題04-4

## 課題04-7